In [1]:
import numpy as np
import os
import time

In [ ]:
# utf 8
import re

standardize_map = {
    'ê': 'e', 'è': 'e', 'é': 'e', 'ë': 'e', 'Ê' : 'E',
    'à': 'a', 'á': 'a', 'â': 'a', 'ä' : 'a', 'ă' :'a',
    'î': 'i', 'ì': 'i', 'í': 'i', 'ï': 'i',
    'ô': 'o', 'ò': 'o', 'ó': 'o', 'ö': 'o',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u'
}

def standardize_text(text):
    for diacritic_char, standard_char in standardize_map.items():
        text = text.replace(diacritic_char, standard_char)
    return text

with open('scrapped.txt', 'r', encoding='utf-8') as file:
    text = file.read()

standardized_text = standardize_text(text).lower()

with open('standardized.txt', 'w', encoding='utf-8') as file:
    file.write(standardized_text)



In [ ]:
# remove punc
import re
import string

def remove_punctuation(text):

    pattern = f"[{re.escape(string.punctuation)}]"
    text_no_punctuation = re.sub(pattern, "", text)
    
    return text_no_punctuation

v2_standardized_text = remove_punctuation(standardized_text)

with open('standardized-2.txt', 'w', encoding='utf-8') as file:
    file.write(v2_standardized_text)


In [4]:
# remove empty lines
def remove_empty_lines(text):
    lines = text.split("\n")
    
    non_empty_lines = [line for line in lines if line.strip()]
    
    cleaned_text = "\n".join(non_empty_lines)
    
    return cleaned_text    


v3_standardized_text = remove_empty_lines(v2_standardized_text)

with open('standardized-3.txt', 'w', encoding='utf-8') as file:
    file.write(v3_standardized_text)

In [23]:
# count
def count_words(text):
    words = text.split()
    return len(words)

word_count = count_words(v2_standardized_text)

print("total :", word_count)


total : 9365


In [5]:
# unique
def get_unique_words(text):
    words = text.split()
    
    unique_words = set(word.lower() for word in words)
    
    return list(unique_words)

unique_words = get_unique_words(v3_standardized_text)
length_of_unique_words = len(unique_words)

print("Jumlah Kata Unik:", len(unique_words))


Jumlah Kata Unik: 2885


In [6]:
# [[],[]]
def text_to_2d_list(text):

    lines = text.split("\n")
    
    words_2d_list = [line.split() for line in lines if line.strip()]
    
    return words_2d_list


words_2d_list = text_to_2d_list(v3_standardized_text)

print(words_2d_list[:2])
# v3_standardized_text


[['lingkup', 'pencarian', 'teks', 'dan', 'catatankakinya', 'teks', 'pencarian', '224', 'karakter', 'filter', 'pencarian', 'huruf', 'besarkecil', 'diakritik', 'serta', 'pungtuasi', 'diabaikan', 'karakter', 'dapat', 'digunakan', 'sebagai', 'pengganti', 'zero', 'atau', 'satu', 'huruf', 'sembarang', 'simbol', 'wildcard', 'dapat', 'digunakan', 'sebagai', 'pengganti', 'zero', 'atau', 'sejumlah', 'karakter', 'termasuk', 'spasi', 'mengakomodasi', 'variasi', 'ejaan', 'antara', 'lain', 'dj', 'j', 'tj', 'c', 'j', 'y', 'oe', 'u', 'd', 'dh', 't', 'th'], ['anggitanipun', 'dawud', 'magang', 'guru', 'ing', 'masaran']]


In [7]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

word_to_idx, idx_to_word = build_indices(unique_words)

In [8]:
# word_to_idx

for i, (word, idx) in enumerate(word_to_idx.items()):
    print(f"{word}: {idx}")
    if i == 9:
        break


dinuta: 0
kumaropok: 1
katiyup: 2
sabe: 3
angasataken: 4
abot: 5
selak: 6
age: 7
amratelakaken: 8
pangaksama: 9


In [36]:
def prepare_corpus(corpus, word_to_idx):
    sequences = []
    
    for line in corpus:
        tokens = line
        missing_tokens = [token for token in tokens if token not in word_to_idx]
        
        for token in missing_tokens:
            word_to_idx[token] = len(word_to_idx)  
            print(f"Token '{token}' ditambahkan ke word_to_idx.")
        
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = [
                word_to_idx[token] for token in i_gram_sequence
            ]
            sequences.append(i_gram_sequence_ids)
    
    return sequences

sequences = prepare_corpus(words_2d_list, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [37]:
sequences[0]

[2564, 2500]

In [38]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

kekesahan
gandum
kasade


In [39]:
len(sequences)

9210

In [40]:
max_sequence_len

146

In [41]:
from keras.preprocessing.sequence import pad_sequences
from keras import utils
import numpy as np

def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = utils.to_categorical(y, length_of_unique_words)
    return X, y

X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)


In [ ]:
from keras.layers import Dense, LSTM, Dropout, Embedding, BatchNormalization
from keras.models import Sequential

def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential()
    
    model.add(Embedding(length_of_unique_words, 64, input_length=max_sequence_len - 1))
    
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(64))
    model.add(Dropout(0.3))
    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(length_of_unique_words, activation='softmax')) 
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'] 
    )
    
    return model


In [52]:
model = create_model(max_sequence_len, length_of_unique_words)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [53]:
len(X)

9210

In [54]:
model.fit(X, y, batch_size = 512, epochs=50)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 759ms/step - accuracy: 0.0041 - loss: 7.8671
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 752ms/step - accuracy: 0.0306 - loss: 7.1070
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 750ms/step - accuracy: 0.0358 - loss: 6.9951
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 753ms/step - accuracy: 0.0321 - loss: 6.9294
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 754ms/step - accuracy: 0.0336 - loss: 6.8982
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 750ms/step - accuracy: 0.0314 - loss: 6.8231
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 749ms/step - accuracy: 0.0354 - loss: 6.7101
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 853ms/step - accuracy: 0.0348 - loss: 6.6138
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 752ms/step - accuracy: 0.0312 - loss: 6.5098
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 726ms/step - accuracy: 0.0307 - loss: 6.4643
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 725ms/step - accuracy: 0.0332 - loss: 6.3359
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14

In [55]:
# def generate_text(seed_text, next_words, model, max_seq_len):
#     for _ in range(next_words):
#         sequences= prepare_corpus(words_2d_list[2], word_to_idx)
#         sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len-1, padding='pre')
#         predicted = model.predict_classes(sequences, verbose=0)
#         output_word = ''
#         output_word = idx_to_word[predicted[0]]            
#         seed_text = seed_text + " " + output_word
        
#     return seed_text.title()

from keras.preprocessing.sequence import pad_sequences

def generate_text(seed_text, next_words, model, max_seq_len, word_to_idx, idx_to_word):
    for _ in range(next_words):
        tokens = seed_text.split()
        token_sequence = [word_to_idx.get(token, 0) for token in tokens] 
        
        padded_sequence = pad_sequences([token_sequence], maxlen=max_seq_len - 1, padding='pre')
        
        predicted_index = np.argmax(model.predict(padded_sequence, verbose=0))
        predicted_word = idx_to_word[predicted_index]
        
        seed_text += f" {predicted_word}"
    
    return seed_text


In [56]:
print(generate_text("kacariyos ing jaman", 30, model, max_sequence_len, word_to_idx, idx_to_word))

kacariyos ing jaman kina sampun mindhak gambira panganggepiun ing kawula kaluhuran sabda ing ing salebeting wardaya namung dados nyenyadhang kalangkunglangkung pendah kalih kalih boten badhe sa endah boten sampun dangu ingkang tata saben


In [50]:
model_structure = model.to_json()
with open("v1_text_generation.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("v1_text_generation.weights.h5")